In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import torchvision.models as models
from torch.utils.data import Dataset as Dataset
import torch.optim.lr_scheduler as lr_scheduler
import time
from PIL import Image

In [ ]:
import torch
from torchvision import datasets, transforms, models, utils

In [ ]:
train_path = '/content/drive/MyDrive/206/Bali Leaf Project/data/train'
val_path = '/content/drive/MyDrive/206/Bali Leaf Project/data/val'
test_path = '/content/drive/MyDrive/206/Bali Leaf Project/data/test'

In [ ]:

train_transformations = transforms.Compose([
    transforms.Resize(320),
    transforms.CenterCrop(300),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(degrees=(-90, 90)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transformations = transforms.Compose([
    transforms.Resize(320),
    transforms.CenterCrop(300),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
train_set = datasets.ImageFolder(train_path, transform = train_transformations)
val_set = datasets.ImageFolder(val_path, transform = transformations)
test_set = datasets.ImageFolder(test_path, transform = transformations)

In [ ]:
print(len(train_set))
print(len(val_set))
print(len(test_set))

In [ ]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    print('img size: ', img.shape)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

# show images
imshow(utils.make_grid(images))
# print labels
# print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

In [ ]:
model = models.resnet18(pretrained=True)

num_labels = 26

classifier = nn.Linear(512, num_labels)

model.fc = classifier

In [ ]:
model

In [ ]:
train_losses = []
train_accuracies = []
val_accuracies = []
val_losses = []

def train_model(model, loss_fn, optimizer, train_loader, val_loader, batch_size, num_epochs, stat_count=100, device=None):
    print('Training...')
    model.to(device)
    min_valid_acc = 0
    # Iterate through all Epochs
    for epoch in range(num_epochs):
        train_loss = 0.0
        val_loss = 0.0
        val_acc = 0.0

        # Iterate through training dataset
        model.train()
        for i, data in enumerate(train_loader, 0):
            # Flatten images and load images/labels onto GPU
            images, labels = data[0].to(device), data[1].to(device)
            # Zero collected gradients at each step
            optimizer.zero_grad()
            # Forward Propagate
            outputs = model(images)
            # Calculate Loss
            loss = loss_fn(outputs, labels)
            # Back Propagate
            loss.backward()
            # Update Weights
            optimizer.step()
            # Get Running Loss
            train_loss += loss.item()
            # Print statistics on every stat_count iteration
            if (i+1) % stat_count == 0:
                print('\tEpoch [%d/%d], Step [%d/%d], Loss: %.4f'
                            %(epoch+1, num_epochs, i+1,
                            len(train_loader), train_loss / i))

        # val dataset
        model.eval()
        with torch.no_grad():
          for inputs, labels in val_loader:
            images, labels = inputs.to(device), labels.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            val_loss += loss.item() #* images.size(0)

        # Print stats
        ep_train_loss = train_loss / len(train_loader)
        ep_val_loss = val_loss / len(val_loader)
        ep_val_acc = test_accuracy(model, val_loader, device=device)
        ep_train_acc = test_accuracy(model, train_loader, device=device)
        print('Epoch [%d/%d], Train (Loss: %.4f, Acc: %.4f), Val (Loss: %.4f, Acc: %.4f), '
                            %(epoch+1, num_epochs, ep_train_loss, ep_train_acc, ep_val_loss, ep_val_acc), end=" | ")
        
        train_losses.append(ep_train_loss)
        train_accuracies.append(ep_train_acc)
        val_accuracies.append(ep_val_acc)
        val_losses.append(ep_val_loss)

def test_accuracy(model, test_loader, device=None):
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for test_data in test_loader:
            images, labels = test_data[0].cuda(), test_data[1].cuda()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

def plot_learning_curve(train_losses, train_accuracies, val_losses, val_accuracies, num_epochs):
  iterations = range(0, num_epochs)
  plt.figure()
  plt.subplot(2,1,1)
  plt.plot(iterations, train_losses, 'r')
  plt.plot(iterations, val_losses, 'b')
  plt.title('Training Performance')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')

  plt.subplot(2,1,2)
  plt.plot(iterations, train_accuracies, 'r')
  plt.plot(iterations, val_accuracies, 'b')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.savefig('LR_curve.png')

In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available else torch.device('cpu')
lr = 1e-3
num_epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

# lambda1 = lambda epoch: epoch / 10
# scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)

In [ ]:
# model.load_state_dict(torch.load('/content/drive/MyDrive/206/Bali Leaf Project/checkpoint_model.pth'))
train_model(model, criterion, optimizer, train_loader, val_loader, batch_size, num_epochs, stat_count=100, device=device)

In [ ]:
# Load best train weights
model.load_state_dict(torch.load('/content/drive/MyDrive/206/Bali Leaf Project/checkpoint_model.pth'))
test_acc = test_accuracy(model, test_loader, device)
print('\nTest Accuracy: ', test_acc)